In [1]:
import sympy as sym
from IPython.display import display

def cubic_spline(xs: list[float], ys: list[float]) -> list[sym.Symbol]:

    # Ordenamos los puntos por la coordenada x
    puntos = sorted(zip(xs, ys), key=lambda x: x[0])  

    xs = [x for x, _ in puntos]
    ys = [y for _, y in puntos]

    n = len(puntos) - 1  # número de splines

    # Distancias entre los puntos consecutivos
    h = [xs[i + 1] - xs[i] for i in range(n)]  

    # Calcular el vector alpha
    alpha = [0]  # valor inicial en alpha[0], no se utiliza
    for i in range(1, n):
        alpha.append(3 / h[i] * (ys[i + 1] - ys[i]) - 3 / h[i - 1] * (ys[i] - ys[i - 1]))

    # Inicializar l, u, z
    l = [1]  # Primer valor de l
    u = [0]  # Primer valor de u
    z = [0]  # Primer valor de z

    # Resolver el sistema tridiagonal
    for i in range(1, n):
        l.append(2 * (xs[i + 1] - xs[i - 1]) - h[i - 1] * u[i - 1])
        u.append(h[i] / l[i])
        z.append((alpha[i] - h[i - 1] * z[i - 1]) / l[i])

    l.append(1)
    z.append(0)

    # Inicializar los coeficientes del spline
    c = [0] * (n + 1)
    b = [0] * n
    d = [0] * n
    a = [0] * n

    # Resolver para los coeficientes c, b, d, y a
    for j in range(n - 1, -1, -1):
        c[j] = z[j] - u[j] * c[j + 1]
        b[j] = (ys[j + 1] - ys[j]) / h[j] - h[j] * (c[j + 1] + 2 * c[j]) / 3
        d[j] = (c[j + 1] - c[j]) / (3 * h[j])
        a[j] = ys[j]

    # Variable simbólica x para la representación del spline
    x = sym.Symbol("x")
    splines = []

    # Construir las expresiones simbólicas de los splines
    for j in range(n):
        S = a[j] + b[j] * (x - xs[j]) + c[j] * (x - xs[j])**2 + d[j] * (x - xs[j])**3
        splines.append(S)

    return splines

xs = [0, 1, 2]
ys = [-5, -4, 3]

splines = cubic_spline(xs=xs, ys=ys)
_ = [display(s) for s in splines]
print("______")
_ = [display(s.expand()) for s in splines]

1.5*x**3 - 0.5*x - 5

4.0*x - 1.5*(x - 1)**3 + 4.5*(x - 1)**2 - 8.0

______


1.5*x**3 - 0.5*x - 5

-1.5*x**3 + 9.0*x**2 - 9.5*x - 2.0

In [4]:
import sympy as sym
from IPython.display import display

def cubic_spline_clamped(
    xs: list[float], ys: list[float], d0: float, dn: float
) -> list[sym.Symbol]:

    # Ordenamos los puntos por la coordenada x
    puntos = sorted(zip(xs, ys), key=lambda x: x[0])  
    xs = [x for x, _ in puntos]
    ys = [y for _, y in puntos]

    n = len(puntos) - 1  # número de splines
    h = [xs[i + 1] - xs[i] for i in range(n)]  # distancias entre los puntos consecutivos

    # Precalculamos el vector alpha
    alpha = [0] * (n + 1)
    alpha[0] = 3 / h[0] * (ys[1] - ys[0]) - 3 * d0
    alpha[n] = 3 * dn - 3 / h[n - 1] * (ys[n] - ys[n - 1])

    for i in range(1, n):
        alpha[i] = 3 / h[i] * (ys[i + 1] - ys[i]) - 3 / h[i - 1] * (ys[i] - ys[i - 1])

    # Inicializamos los vectores l, u, z
    l = [2 * h[0]]
    u = [0.5]
    z = [alpha[0] / l[0]]

    for i in range(1, n):
        l.append(2 * (xs[i + 1] - xs[i - 1]) - h[i - 1] * u[i - 1])
        u.append(h[i] / l[i])
        z.append((alpha[i] - h[i - 1] * z[i - 1]) / l[i])

    l.append(h[n - 1] * (2 - u[n - 1]))
    z.append((alpha[n] - h[n - 1] * z[n - 1]) / l[n])

    # Inicializamos el vector c
    c = [0] * (n + 1)
    c[-1] = z[-1]  # La última entrada de c es z[n]

    # Coeficientes a, b, c, d para los splines
    b = [0] * n
    d = [0] * n
    a = [0] * n

    # Calculamos los coeficientes para cada spline
    for j in range(n - 1, -1, -1):
        c[j] = z[j] - u[j] * c[j + 1]
        b[j] = (ys[j + 1] - ys[j]) / h[j] - h[j] * (c[j + 1] + 2 * c[j]) / 3
        d[j] = (c[j + 1] - c[j]) / (3 * h[j])
        a[j] = ys[j]

    # Variable simbólica x para la expresión de los splines
    x = sym.Symbol("x")
    splines = []

    # Construimos las expresiones simbólicas para cada spline
    for j in range(n):
        S = a[j] + b[j] * (x - xs[j]) + c[j] * (x - xs[j]) ** 2 + d[j] * (x - xs[j]) ** 3
        splines.append(S)

    return splines

# Ejemplo de uso:
xs = [0, 1, 2]
ys = [0, 1, 0]
d0 = 0  # Derivada en el primer punto
dn = 0  # Derivada en el último punto

# Llamada a la función
splines = cubic_spline_clamped(xs, ys, d0, dn)

# Mostrar los resultados
for spline in splines:
    display(spline)

-2.0*x**3 + 3.0*x**2

2.22044604925031e-16*x + 2.0*(x - 1)**3 - 3.0*(x - 1)**2 + 1.0

In [5]:
# Definir los puntos y las derivadas en los extremos
xs = [1, 2, 3]
ys = [2, 3, 5]
d0 = 0  # Derivada en el primer punto (puedes ajustarlo si es necesario)
dn = 0  # Derivada en el último punto (puedes ajustarlo si es necesario)

# Llamada a la función cubic_spline_clamped
splines = cubic_spline_clamped(xs, ys, d0, dn)

# Mostrar los resultados
for spline in splines:
    display(spline)

0.25*(x - 1)**3 + 0.75*(x - 1)**2 + 2

2.25*x - 1.75*(x - 2)**3 + 1.5*(x - 2)**2 - 1.5